In [2]:
import pandas as pd
import numpy as np
import sys, os

from matplotlib import pyplot as plt
sys.path.insert(1, os.path.join(sys.path[0], '../../'))  # for importing local packages from src
sys.path.insert(1, os.path.join(sys.path[0], '../'))  # for importing model config

%reload_ext autoreload
%autoreload 2 

## Datasets

In [5]:
from src.model.rc_dataset import ExperimentDataset


DATASET_NEMA_H0 = '../datasets/dataset_NEMA_NEMA_H0.csv'
DATASET_NEMA_H3 = '../datasets/dataset_NEMA_NEMA_H3.csv'
DATASET_NEMA_H15 = '../datasets/dataset_NEMA_NEMA_H15.csv'

dataset_nema_h0 = ExperimentDataset(csv_path=DATASET_NEMA_H0)
dataset_nema_h3 = ExperimentDataset(csv_path=DATASET_NEMA_H3)
dataset_nema_h15 = ExperimentDataset(csv_path=DATASET_NEMA_H15)

datasets = [
  dataset_nema_h0, 
  dataset_nema_h3,
  dataset_nema_h15
]

## Model configuration

In [7]:
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.model_selection import LeaveOneGroupOut
from src.learning.scorers import nmse_scorer
from pipeline_cnwheat import GroupGenerator, TimeGenerator
from pipeline_base import TrainTestSplitter, DirectTransform, WarmupTransform


readout_model = Pipeline([
  ('ridge_regression', Ridge(alpha=1, fit_intercept=True))
])

model_param_grid = [{
  'ridge_regression__alpha': 10 ** np.linspace(np.log10(1e-4), np.log10(1e2), 50)
}]

warmup_days = 4

shared_pipeline_params = {
  # Data generation
  'datasets': datasets,
  'groups': GroupGenerator(day_length=24),
  'time': TimeGenerator(day_length=24),
  
  # Model training and validation
  'readout_model': readout_model,
  'model_param_grid': model_param_grid,
  'model_scorer': nmse_scorer,
  'folds': LeaveOneGroupOut(),
  'train_test_split': TrainTestSplitter(block_size=4, test_ratio=0.5),
}

## Regression Pipelines

In [8]:
from pipeline_base import (
  RCPipeline, 
  Rescale, 
  DaylightMask
)

from pipeline_cnwheat import (
  TargetGenerator, 
  SingleReservoirGenerator,
  MultiReservoirGenerator,
  TargetReservoirGenerator,
  GroupRescale
)

from model_config_cnwheat import (
  baseline_reservoirs,
  heterogeneous_reservoirs, 
  targets, 
  measurable_reservoirs
)

### Input and physiological task regressions

In [11]:
shared_params = {
  'transforms': [
    WarmupTransform(warmup_days=4, day_length=24),
    DirectTransform()
  ]
}

In [ ]:
target_state_pairs = [(target, state_var) for target in targets for state_var in measurable_reservoirs]
target_env_pairs = [(name, target, env_targets) for target in targets for (name, env_targets) in measurable_reservoirs]
target_het_pairs = [(name, target, state_vars) for target in targets for (name, state_vars) in heterogeneous_reservoirs]